In [1]:
import numpy as np

import warnings
warnings.filterwarnings("ignore", category=Warning)

import paddle
import paddle.fluid as fluid
from paddle.static import InputSpec
from paddle.fluid.framework import core
from paddle.vision import image_load

import matplotlib.pyplot as plt
import functools

from collections import Counter
import os
import gc
from collections import Counter
from itertools import repeat
import time
import tqdm

PLEASE USE OMP_NUM_THREADS WISELY.


In [2]:
learning_rate = 1e-2

batch_size = 32
samples = 260000

test_samples = 10526
n_class = 14
training_epochs = 20

image_size = 224

n_C = 0.1
n_V = 0.01
n_R = 0.02
t_w = 1

In [3]:
class Dataset(paddle.io.Dataset):
    def __init__(self, num_samples, dir_, save_path, transform):
        super(Dataset, self).__init__()
        self.num_samples = num_samples
        self.transform = transform
        self.dir_ = dir_
        self.data_dir = []
        self.label_list = []
        with open(self.dir_) as f:
            for line in f:
                temp_dir, label = line.split()
                self.data_dir.append(temp_dir)
                self.label_list.append(int(label))
        
        self.labels = np.array([l for l in self.label_list])
        np.save(save_path, self.labels)

    def __getitem__(self, index):
        f_path = self.data_dir[index].split('/')
        dir_ = r'/root/clothing1m'
        path = os.path.join(dir_,*f_path)
        image = image_load(path)
#         image = jpeg.JPEG(path).decode()
        data = np.array(image)
        if len(data.shape) == 2:
            data = np.expand_dims(data, axis=-1)
            data = np.tile(data, (1,1,3))
        if self.transform is not None:
            data = self.transform(data)
        return index, data
    def __len__(self):
        return self.num_samples
    

In [4]:
import paddle.vision.transforms as T

mean,std = ((0.6959, 0.6537, 0.6371), (0.3113, 0.3192, 0.3214))
mean = np.array(mean).reshape(1,1,3)*255
std = np.array(std).reshape(1,1,3)*255

transform_robust = T.Compose([
                    T.RandomHorizontalFlip(),
                    T.Resize(256),
                    T.RandomCrop(image_size),
                    T.Transpose(order=(2,1,0,)),
                    T.Normalize(mean=mean,std=std)
                    ])
transform_show = T.Compose([
                    T.Resize((image_size, image_size)),
                    T.Transpose(order=(2,1,0,)),
                    ])
transform_clean = T.Compose([
#                     T.ColorJitter(),
                    T.Resize((256, 256)),
                    T.CenterCrop((image_size, image_size)),
                    T.Transpose(order=(2,1,0,)),
                    T.Normalize(mean=mean,std=std)
                    ])

print('Start download training data and load training data.')
train_dataset = Dataset(samples, 'pseudo_noisy_train_key_list_kv.txt', 'train_Y_noised_pseudo.npy', transform_robust)
testclean_dataset = Dataset(test_samples, 'clean_test_key_list_kv.txt', 'testclean_Y.npy', transform_clean)
train_loader = paddle.io.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers = 6)
test_loader = paddle.io.DataLoader(testclean_dataset, batch_size=128, shuffle=False, num_workers = 6)


train_Y_noised = np.load('train_Y_noised_pseudo.npy')[:samples]
train_Y = train_Y_noised
test_Y = np.load('testclean_Y.npy')[:test_samples]
Yt_list = [train_Y_noised]
print('Finished.')


Start download training data and load training data.
Finished.


In [5]:
def validation():
    loader = test_loader()
    loss_eval = 0
    acc_eval = 0
    network_3.eval()
    Py_temp = np.zeros((test_samples,),dtype = np.float32)
    Feature_temp = np.zeros((test_samples,network_3.fc.weight.shape[0]),dtype = np.float32)
    with paddle.no_grad():
        for batch_id, (index,X_data) in enumerate(loader):
            Y_data = np.array(test_Y[index]).astype(np.int64)
            temp_X = paddle.to_tensor(X_data)
            Y_GPU = paddle.to_tensor(Y_data)
            y_onehot = paddle.nn.functional.one_hot(paddle.reshape(Y_GPU,(-1,)),num_classes=n_class)

            first = network_3.maxpool(network_3.relu(network_3.bn1(network_3.conv1(temp_X))))
            l1 = network_3.layer1(first)
            l2 = network_3.layer2(l1)
            l3 = network_3.layer3(l2)
            l4 = network_3.layer4(l3)
            feature = paddle.reshape(network_3.avgpool(l4), (l4.shape[0],-1))

            logits = network_3.fc(feature)
            probs = paddle.nn.functional.softmax(logits)
            Py = paddle.sum(y_onehot * probs, axis = -1)
            Py_temp[index] = Py.numpy()
            Feature_temp[index] = feature.numpy()
            
            loss = loss_fn(logits, paddle.reshape(Y_GPU,(-1,1)))
            
            acc = paddle.metric.accuracy(logits, paddle.reshape(Y_GPU,(-1,1)))

            loss_eval += loss.numpy()
            acc_eval += acc.numpy()

        loss_eval/=(batch_id+1)
        acc_eval/=(batch_id+1)
    return loss_eval, acc_eval, Feature_temp

In [6]:

logits_list = []
Pred_list = []

acc_list = []
loss_list = []
loss_train_list = []

Py_list = [np.random.rand(samples,)]
score_list = [np.random.rand(samples,)]

loss_fn = paddle.nn.CrossEntropyLoss()
network_3 = paddle.vision.models.resnet50(num_classes=n_class,pretrained=True)


scheduel = paddle.optimizer.lr.MultiStepDecay(learning_rate=learning_rate, milestones=[3, 6, 9], gamma=0.1)

opt = paddle.optimizer.Momentum(learning_rate = scheduel,
                                parameters=network_3.parameters(),)
warnings.filterwarnings("ignore", category=Warning) 

W0504 09:48:47.635505  9877 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 8.6, Driver API Version: 11.7, Runtime API Version: 11.2
W0504 09:48:47.639564  9877 gpu_resources.cc:91] device: 0, cuDNN Version: 8.1.


In [ ]:
start_time = time.time()

for epoch_id in range(training_epochs):
    network_3.train()
    feature_num = network_3.fc.parameters()[0].shape[0]
    
    loader = train_loader()

    loss_train = 0
    acc_train = 0
    acc_train_ori = 0
    loss_train_ori = 0
    Py_temp = np.zeros((samples,),dtype=np.float32)
    Pred_temp = np.zeros((samples,),dtype=np.float32)
    Pred_temp_other = np.zeros((samples,),dtype=np.float32)
    Probs_temp = np.zeros((samples,n_class),dtype=np.float32)
    Logits_temp = np.zeros((samples,n_class),dtype=np.float32)
    Logits_temp_other = np.zeros((samples,n_class),dtype=np.float32)
    feature_temp = np.zeros((samples, feature_num), dtype = np.float32)
    Pred_onehot_temp = np.ones((samples,n_class),dtype=np.float32)  
    
    thres = np.sort(Py_list[-1])[int(samples*n_R)]    
    relabel_mask = Py_list[-1] <= thres   

    Y_onehot = np.eye(n_class)[Yt_list[-1]]
    Y_onehot_0 = np.eye(n_class)[Yt_list[0]]
    for batch_id, (index, X_data) in enumerate(loader):
        Y_data = np.array(Yt_list[-1][index]).astype(np.int64)
        Y_data_ori = np.array(train_Y[index]).astype(np.int64)
        pred_onehot_temp = Pred_onehot_temp[index]
                
        temp_X = paddle.to_tensor(X_data)
        Y_GPU = paddle.to_tensor(Y_data)
        Y_GPU_ori = paddle.to_tensor(Y_data_ori)
        y_onehot = paddle.nn.functional.one_hot(paddle.reshape(Y_GPU,(-1,)),num_classes=n_class)

        first = network_3.maxpool(network_3.relu(network_3.bn1(network_3.conv1(temp_X))))
        l1 = network_3.layer1(first)
        l2 = network_3.layer2(l1)
        l3 = network_3.layer3(l2)
        l4 = network_3.layer4(l3)
        feature = paddle.reshape(network_3.avgpool(l4), (l4.shape[0],-1))

        logits = network_3.fc(feature)

        probs = paddle.nn.functional.softmax(logits)
        Py = paddle.sum(y_onehot * probs, axis = -1)
        Pred = paddle.argmax(probs,axis=-1)
        logits_other = logits - y_onehot*1e10
        Pred_other = paddle.argmax(logits_other,axis=-1)

        if epoch_id < t_w:
            loss = loss_fn(logits,paddle.reshape(Y_GPU,(-1,1)))
        else:
            Y_GPU = paddle.where(paddle.to_tensor(relabel_mask[index]), Pred_other, Y_GPU)
            loss = loss_fn(logits,paddle.reshape(Y_GPU,(-1,1)))

            
        Py_temp[index] = Py.numpy()
        Pred_temp[index] = Pred.numpy()

        Probs_temp[index] = probs.numpy()
        Logits_temp[index] = logits.numpy()
        feature_temp[index] = feature.numpy()
                
        
        acc = paddle.metric.accuracy(logits, paddle.reshape(Y_GPU,(-1,1)))
        acc_ori = paddle.metric.accuracy(logits, paddle.reshape(Y_GPU_ori,(-1,1)))
        loss_ori = loss_fn(logits,paddle.reshape(Y_GPU_ori,(-1,1)))

        loss_train += loss.numpy()
        acc_train += acc.numpy()
        acc_train_ori += acc_ori.numpy()
        loss_train_ori += loss_ori.numpy()
        
        loss.backward()
        
        opt.step()
        opt.clear_grad()
        if batch_id%int(4000/batch_size)==0:
            _acc = np.sum(Pred.numpy().ravel()==Y_data.ravel(),axis=-1)/Y_data.shape[0]  
            _loss = loss.numpy()
            end_time =time.time()
            print('batch_id:%4d, train loss :%.4f, acc:%.4f, time elapsed %4d'%(batch_id, _loss, _acc, end_time - start_time), flush=True)
    scheduel.step()
        
    loss_train/=(batch_id+1)
    acc_train/=(batch_id+1)
    acc_train_ori/=(batch_id+1)
    loss_train_ori/=(batch_id+1)

    print('epoch %d train complete'%epoch_id)
    loss_eval, acc_eval, feature_val = validation()

    loss_train_list.append(loss_train)

    loss_list.append(loss_eval)
    acc_list.append(acc_eval)
    Pred_list.append(Pred_temp)
    Py_list.append(Py_temp)


    logits_list.append(Logits_temp)
    end_time = time.time()

    print_str = 'train loss:%.4f, train acc:%.4f, train acc ori:%.4f,  eval loss:%.4f, eval acc:%.4f, time elapsed:%.4f'%(loss_train, acc_train, acc_train_ori, loss_eval, acc_eval, end_time - start_time)
    print(print_str)  
    with open('log.txt', 'a') as f:
        f.write(print_str + '\n')

    anchor_points = []
    anchor_mask = np.zeros((samples,),dtype=bool)
    Py_mean = np.zeros((samples,))

    for j in range(len(Py_list)):
        Py_mean+=Py_list[j]
    Py_mean/=len(Py_list)   

    for j in range(n_class):
        class_mask = Yt_list[0] == j
        c_n = class_mask.sum()
        c_th = np.sort(Py_mean[class_mask])[-int(c_n * n_C)]
        anchor_points.append(np.where(np.logical_and(Py_mean>=c_th,class_mask))[0])
    anchor_points = np.concatenate(anchor_points)
    anchor_mask[anchor_points] = True

    L_batch = 1000
    Y_onehot = np.eye(n_class)[Yt_list[-1]]
    zC = paddle.to_tensor(feature_temp[anchor_points])
    fC = paddle.to_tensor(Probs_temp[anchor_points])
    yC = paddle.to_tensor(Y_onehot[anchor_points])
    fCcyC = fC - yC
    lr = 1e-6
    learning_risk = np.zeros((samples,)) 
    for j in range(int(np.ceil(samples/L_batch))):
        i_ind = np.arange(j*L_batch, min(samples,(j+1)*L_batch))
        zi = paddle.to_tensor(feature_temp[i_ind])
        fi = paddle.to_tensor(Probs_temp[i_ind])
        yi = paddle.to_tensor(Y_onehot[i_ind])
        zixzC = zi @ zC.transpose([1,0])
        part_1_1 = (zixzC + 1) @ fCcyC
        part1 = (part_1_1 * (yi-fi)).sum(axis=-1,keepdim=True)*4*lr/len(anchor_points)

        learning_risk[i_ind] = part1.numpy().ravel()

    select = np.zeros((samples,),dtype=np.bool8)
    score = learning_risk
    for j in range(n_class):
        class_mask = Yt_list[-1] == j
        c_n = class_mask.sum()
        c_th = np.sort(score[class_mask])[-min(int(c_n * n_V),c_n)]
        select[np.logical_and(score>=c_th, class_mask)] = True
    if epoch_id < t_w:
        temp_Yt_noised = Yt_list[-1]
    else:
        temp_Yt_noised=np.where(select.ravel(), Pred_temp.ravel(), Yt_list[-1].ravel()).astype(int)

    score_list.append(score)
    